# Imports

In [ ]:
import dspy
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCase, LLMTestCaseParams
import os

# Configs

In [2]:
os.environ["OPENAI_API_KEY"] = ""

# DSPy Signatures

In [3]:
class SimpleQA(dspy.Signature):
    """Answer a simple question with a simple answer in markdown format."""
    question = dspy.InputField()
    answer = dspy.OutputField(desc="formatted in markdown with ## headers and bullet points")

In [4]:
class RetryMarkdownAnswer(dspy.Signature):
    """Correct the answer to use proper markdown format with headers (##) and bullet points (-)."""
    original_answer = dspy.InputField()
    reason_for_failure = dspy.InputField()
    corrected_answer = dspy.OutputField()


# Deep Eval Metric

In [5]:
explainability_metric = GEval(
    name="ApplicationExplainability",
    criteria="Verify that the answer has explained the applications of the technology mentioned in 'input'",
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
    evaluation_steps=[
        "Check whether the inventor of the technology mentioned"
    ],
)

In [6]:
class RetryMarkdownQA(dspy.Module):
    def __init__(self):
        self.generate = dspy.Predict(SimpleQA)
        self.retry = dspy.Predict(RetryMarkdownAnswer)
        
    def forward(self, question):
        answer = self.generate(question=question).answer
        retry_count = 0
        
        while retry_count < 3:
            test_case = LLMTestCase(
                input=question,
                actual_output=answer
            )
            
            explainability_metric.measure(test_case)
            
            if explainability_metric.score >= 0.5:
                return answer

            print("**** Retrying ....... ****")
            answer = self.retry(
                original_answer=answer,
                reason_for_failure=explainability_metric.reason
            ).corrected_answer
            
            retry_count += 1
            print(f"Retry attempt {retry_count}. Reason: {explainability_metric.reason}")
            
        return answer

# Run Flow

In [7]:
qa_system = RetryMarkdownQA()
question = "Explain quantum computing basics"

lm = dspy.LM('openai/gpt-4o-mini')
dspy.configure(lm=lm)

response = qa_system(question)
print(f"\nFinal Answer:\n{response}")

Output()

**** Retrying ... ****


Output()

Retry attempt 1. Reason: The output does not mention the inventor of quantum computing or any specific individual associated with its invention.



Final Answer:
## Quantum Computing Basics

- **Definition**: Quantum computing is a type of computation that uses quantum bits (qubits) instead of classical bits to process information.
  
- **Qubits**: 
  - Unlike classical bits, which can be either 0 or 1, qubits can exist in multiple states simultaneously due to superposition.
  
- **Superposition**: 
  - This property allows qubits to perform many calculations at once, potentially leading to faster problem-solving capabilities.

- **Entanglement**: 
  - Qubits can be entangled, meaning the state of one qubit is directly related to the state of another, no matter the distance between them. This can enhance computational power.

- **Quantum Gates**: 
  - Quantum operations are performed using quantum gates, which manipulate qubits through various transformations.

- **Applications**: 
  - Quantum computing has potential applications in cryptography, optimization problems, drug discovery, and complex simulations.

- **Pioneers**: 
  